## Data Cleaning Plan

We have three data sets:
- Markers' bios and metadata (markers_bios)
- Followers' bios and metadata (followers_bios)
- All brands and their followers (markers-followers)


Step by step plan:
1. Load the bios of followers, and the marker-follower file. 
    - Provide summary statistics of users and brands. How many brands do we have? How many followers? Any missing data, duplicates etc.?

2. Filter on marker-follower df:
    - Create a dictionary of counts brands per follower
    - Remove users that follow less than 5 (or more) brands
    - Continuously track numbers of users removed
    - Match the Follower_Ids in the now filtered marker-follower df with the follower-bio df. As such, the follower bios will only include users that follow more than five brands. Subsequent filters will be on the correct users (up to date follower-bios).

3. Do the filters on the follower-bios:
    - Remove users with less than 25 followers
    - Remove users with less than 100 tweets

4. Filter based on language: keep only french accounts









In [3]:
# Standard library imports
from collections import defaultdict
from datetime import datetime
import csv
import html
import os
import re
import sys

# Third-party library imports
import dask.dataframe as dd
from joblib import Parallel, delayed
from langdetect import detect_langs, LangDetectException, DetectorFactory
import matplotlib.pyplot as plt
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import pandas as pd
import psutil
import pickle
import regex
import seaborn as sns
from unidecode import unidecode
import gcld3
import ftfy
import importlib

# Local application/library specific imports
import utils2
from utils2 import *

## 1. Load files and summary stats

In [4]:
# # Load the data files and rename ID columns
importlib.reload(utils2)

# Load markers-followers
#Load marker followers
load_path = '/home/livtollanes/SocialMarkers'
file = 'markers_followers_2023-05-19.csv'

req_cols = ['id', 'follower_id']
dtypes = {'id': 'object',
          'follower_id': 'object'}

markers_followers = utils2.fileloader(load_path, file, req_cols, dtypes)


#rename the twittwer id column to follower id 
markers_followers.rename(columns={'id':'marker_id'}, inplace=True)

In [5]:
# Load the followers bios and rename ID columns
load_path = '/home/livtollanes/SocialMarkers'
file = 'markers_followers_bios_2023-05-19.csv'

req_cols = ['twitter_id', 'id', 'screen_name', 'description', 'location', 'tweets', 'followers', 'friends', 'likes', 'lists','timestamp_utc']

dtypes = {
    'twitter_id': 'object',
    'id': 'object',
    'screen_name': 'object',
    'description': 'object',
    'location': 'object',
    'tweets': 'float64',
    'followers': 'float64',
    'friends': 'float64',
    'witheld_in_countries': 'float64'
}

followers_bios = utils2.fileloader(load_path, file, req_cols, dtypes)



#rename the twittwer id column to follower id 
followers_bios.rename(columns={'twitter_id':'follower_id'}, inplace=True)

Summary statistics

In [6]:
importlib.reload(utils2)
utils2.summary_stats(followers_bios, print_dtypes=False)

Shape of DataFrame:  (70666646, 11)

Columns in DataFrame:  ['follower_id', 'id', 'screen_name', 'description', 'timestamp_utc', 'location', 'tweets', 'followers', 'friends', 'likes', 'lists']

Number of unique values in 'follower_id':  70666646

Number of duplicate values in 'follower_id':  0

Number of unique values in 'id':  70642661

Number of duplicate values in 'id':  23984

Number of missing values in each column:
 follower_id             0
id                  23985
screen_name         23986
description      42027215
timestamp_utc       23985
location         47956041
tweets              23985
followers           23985
friends             23985
likes               23985
lists               23985
dtype: int64


In [7]:
importlib.reload(utils2)
utils2.summary_stats(markers_followers, print_dtypes=False)

Shape of DataFrame:  (126345412, 2)

Columns in DataFrame:  ['marker_id', 'follower_id']

Number of unique values in 'follower_id':  70636295

Number of duplicate values in 'follower_id':  55709117

Number of unique values in 'marker_id':  236

Number of duplicate values in 'marker_id':  126345176

Number of missing values in each column:
 marker_id      0
follower_id    0
dtype: int64


In [8]:
compare_column_values(followers_bios, markers_followers, 'follower_id')

There are 30351 unique values in df1 that don't exist in df2.
There are 0 unique values in df2 that don't exist in df1.


## 2. Filter the marker-follower df

- Filter the marker-follower df:
    - Remove users that follow less than 5 (or more) brands

    - Continuously track numbers of users removed
    
    - Match the Follower_Ids in the now filtered marker-follower df with the follower-bio df. As such, the follower bios 
    will only include users that follow more than five brands. Subsequent filters will be on the correct users (up to date follower-bios).

Remove users that follow less than 5 brands

In [15]:
n = 5  # minimal number of brands followed required to be included in the analysis
importlib.reload(utils2)
markers_followers_5 = utils2.filter_followers(markers_followers, 'follower_id', n)

66693204 followers follow less than 5 brands (94.42% of the total followers).
After removing these followers, 3943091 followers are left (5.58% of the followers in the inputted df).


In [19]:
#number of unique brands left
#markers_followers_5['marker_id'].nunique()

# Get the unique marker_id values in the original and filtered DataFrames
original_brands = set(markers_followers['marker_id'].unique())
filtered_brands = set(markers_followers_5['marker_id'].unique())

# Find the brands that are in the original DataFrame but not in the filtered DataFrame
removed_brands = original_brands - filtered_brands

# Print the removed brands
print("Removed brands:", removed_brands) #corresponds to "Napapijiri97", which kindof sounds like a fake profile


Removed brands: {'1059975643'}


Match the IDs in the filtered marker-follower df with the follower bio df, so that the follower bios only are for those who follow at least 5 brands

In [20]:
followers_bios_5 = utils2.streamline_IDs(markers_followers_5, followers_bios, 'follower_id')

Number of unique follower_id in source DataFrame: 3943091
Number of unique follower_id in filtered DataFrame after filtering: 3943091
Removed 66723555 rows from the DataFrame to be filtered.
3943091 rows are left in the filtered DataFrame.


In [21]:
compare_column_values(followers_bios_5, markers_followers_5, 'follower_id')   

There are 0 unique values in df1 that don't exist in df2.
There are 0 unique values in df2 that don't exist in df1.


## 3. Do the filters on the follower-bios:
- Remove users with less than 25 followers
- Remove users with less than 100 tweets
- Update the markers-followers df to match the now filtered bio df
- Filter based on language: keep only french accounts


In [22]:

followers_bios_fullfilter = utils2.filter_by_tweets_and_followers(followers_bios_5, min_followers= 25, min_tweets= 100)


Removed 2750559 rows.
1192532 rows are left.


Again, remove the follower_Ids in markers-followers that don't occur in the newly filtered  followers_bios_nd5_tweets_followers

In [25]:
markers_followers_fullfilter = utils2.streamline_IDs(source= followers_bios_fullfilter, df_tofilter=markers_followers_5, column='follower_id')

Number of unique follower_id in source DataFrame: 1192532
Number of unique follower_id in filtered DataFrame after filtering: 1192532
Removed 19057990 rows from the DataFrame to be filtered.
9730265 rows are left in the filtered DataFrame.


In [26]:
compare_column_values(followers_bios_fullfilter, markers_followers_fullfilter , 'follower_id')

There are 0 unique values in df1 that don't exist in df2.
There are 0 unique values in df2 that don't exist in df1.


In [28]:
#Before writing ti csv, clean description column to avoid writing problems
importlib.reload(utils2)

followers_bios_fullfilter = utils2.process_description(followers_bios_fullfilter, 'description')

In [32]:
# # #Now write the two dfs to csvs to save them in case something happens
markers_followers_fullfilter.to_csv('/home/livtollanes/NewData/markers_followers_cleaned_nolang.csv', encoding='utf-8', index=False)

followers_bios_fullfilter.to_csv('/home/livtollanes/NewData/followers_bios_cleaned_nolang3.csv', sep=',', encoding='utf-8', index=False, quoting=csv.QUOTE_NONNUMERIC)



## 4. Filter based on language: keep only french accounts
- Use language recognition alorithms to filter the follower_bios. 
- We only want french language bios to be included


In [2]:
#Load marker followers
full_path1 = '/home/livtollanes/NewData/markers_followers_cleaned_nolang.csv'
req_cols = ['marker_id', 'follower_id']
dtypes = {'marker_id': 'object',
          'follower_id': 'object'}

markers_followers_clean = pd.read_csv(full_path1, encoding='utf-8', dtype=dtypes, usecols=req_cols)

In [3]:
# #Loading the followers bios (with cleaned description column)
full_path = '/home/livtollanes/NewData/followers_bios_cleaned_nolang3.csv'

req_cols = ['follower_id', 'screen_name', 'description', 'description_cleantext', 'location', 'tweets', 'followers', 'friends', 'likes', 'lists','timestamp_utc']

dtypes = {
    'follower_id': 'object',
    'screen_name': 'object',
    'description': 'object',
    'description_cleantext': 'object',
    'location': 'object',
    'tweets': 'float64',
    'followers': 'float64',
    'friends': 'float64'
}

follower_bios_cleaned3 = pd.read_csv(full_path, usecols=req_cols, dtype=dtypes, engine= 'python')

In [4]:
compare_column_values(follower_bios_cleaned3, markers_followers_clean, 'follower_id')

#The follower_ids are still streamlined, indicating that writing and reading of the cleaned dfs was successful

There are 0 unique values in df1 that don't exist in df2.
There are 0 unique values in df2 that don't exist in df1.


In [101]:
#should probably do summary stats on the dfs again, to make sure that no strange things have happened during the cleaning process
summary_stats(follower_bios_cleaned3, print_dtypes= True)

Shape of DataFrame:  (1240503, 11)

Columns in DataFrame:  ['follower_id', 'screen_name', 'description', 'timestamp_utc', 'location', 'tweets', 'followers', 'friends', 'likes', 'lists', 'description_cleantext']

Data types of columns:
 follower_id               object
screen_name               object
description               object
timestamp_utc            float64
location                  object
tweets                   float64
followers                float64
friends                  float64
likes                    float64
lists                    float64
description_cleantext     object
dtype: object

Number of unique values in 'follower_id':  1240503

Number of duplicate values in 'follower_id':  0

Number of missing values in each column:
 follower_id                   0
screen_name                   0
description              307212
timestamp_utc                 0
location                 387801
tweets                        0
followers                     0
friends            

In [102]:
summary_stats(markers_followers_clean, print_dtypes= True)

Shape of DataFrame:  (9970120, 2)

Columns in DataFrame:  ['marker_id', 'follower_id']

Data types of columns:
 marker_id      object
follower_id    object
dtype: object

Number of unique values in 'follower_id':  1240503

Number of duplicate values in 'follower_id':  8729617

Number of unique values in 'marker_id':  235

Number of duplicate values in 'marker_id':  9969885

Number of missing values in each column:
 marker_id      0
follower_id    0
dtype: int64


Here comes the language detection

- First language detection is with the langdetect package. This package is based on Google's language detection API. On medium, it is actually reported that this package may not perform very accurate on short or mixed lnaguage texts, which is exactly what we are deailing with in twitter bios. 
- Gcld3 mmight be better at handling short text inputs like bios. By reading the documentation, gcld3 seems to be more fitting for social media data language detection

In [106]:
importlib.reload(utils2)
# Create a copy of the DataFrame for each function
follower_bios_cleaned3_copy1 = follower_bios_cleaned3.copy()
follower_bios_cleaned3_copy2 = follower_bios_cleaned3.copy()

# Use the copied DataFrames in the functions
lang = utils2.add_and_detect_language(follower_bios_cleaned3_copy1, 'description_cleantext', seed = 3, n_jobs=-1)
lang2 = utils2.detect_language_gcld3(follower_bios_cleaned3_copy2, 'description_cleantext', n_jobs=-1)

In [107]:
total_rows = lang.shape[0]

french_percent = (lang[lang['language'] == 'fr'].shape[0] / total_rows) * 100
english_percent = (lang[lang['language'] == 'en'].shape[0] / total_rows) * 100
unknown_percent = (lang[lang['language'] == 'unknown'].shape[0] / total_rows) * 100
other_percent = 100 - french_percent - english_percent - unknown_percent

print("French: ", french_percent, "%")
print("English: ", english_percent, "%")
print("Unknown: ", unknown_percent, "%")
print("Other: ", other_percent, "%")

nan_percent = (lang['description_cleantext'].isna().sum() / total_rows) * 100
print("NaN in description_cleantext: ", nan_percent, "%")

French:  21.732232812012548 %
English:  30.755104985638894 %
Unknown:  27.27240482288233 %
Other:  20.240257379466236 %
NaN in description_cleantext:  26.162371231669734 %


In [108]:
total_rows = lang2.shape[0]

french_percent = (lang2[lang2['language'] == 'fr'].shape[0] / total_rows) * 100
english_percent = (lang2[lang2['language'] == 'en'].shape[0] / total_rows) * 100
unknown_percent = (lang2[lang2['language'] == 'unknown'].shape[0] / total_rows) * 100
other_percent = 100 - french_percent - english_percent - unknown_percent

print("French: ", french_percent, "%")
print("English: ", english_percent, "%")
print("Unknown: ", unknown_percent, "%")
print("Other: ", other_percent, "%")

nan_percent = (lang2['description_cleantext'].isna().sum() / total_rows) * 100
print("NaN in description_cleantext: ", nan_percent, "%")

French:  19.973027070470607 %
English:  22.16471866654091 %
Unknown:  26.162371231669734 %
Other:  31.699883031318752 %
NaN in description_cleantext:  26.162371231669734 %


### Manually creating test sets to compare the two language detection methods

For the langdetect df

In [73]:
test1 = lang.sample(n=100, random_state=1)
test2 = lang2.sample(n=100, random_state=1)

In [86]:
test1.head(5)

,follower_id,screen_name,description,timestamp_utc,location,tweets,followers,friends,likes,lists,description_cleantext,language
700690,501147199,MorganJewelers1,Morgan Jewelers has been making a difference in families for generations (since 1914 to be exact)!,1.330031e+09,NaN,405.0,2957.0,479.0,17.0,7.0,Morgan Jewelers has been making a difference in families for generations (since 1914 to be exact)!,en
840602,457164360,LeighAnnTowne,"Host FOX 17 Morning Mix, jokester, DIY-er, thrifter, craft beer nut, loves Bloody Mary's, Bronson's mom\nhttps://www.facebook.com/leighann.towne/",1.325906e+09,"Grand Rapids, Michigan",4497.0,2373.0,2404.0,1221.0,48.0,"Host FOX 17 Morning Mix, jokester, DIY-er, thrifter, craft beer nut, loves Bloody Mary's, Bronson's momhttps://www.facebook.com/leighann.towne/",en
593572,2922883419,ALBrutel,NaN,1.418652e+09,Paris,644.0,100.0,189.0,187.0,4.0,NaN,unknown
1053450,2422167290,Fonkwadiour1,BAYE FALL,1.396361e+09,"TOUBA, SENEGAL",384.0,38.0,215.0,238.0,0.0,BAYE FALL,de
1175310,2677456479,JUSTINAGROSSO,NaN,1.404611e+09,NaN,11011.0,218.0,129.0,2429.0,3.0,NaN,unknown


In [83]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
test1[['screen_name', 'location','description_cleantext', 'language']]

,screen_name,location,description_cleantext,language
700690,MorganJewelers1,NaN,Morgan Jewelers has been making a difference in families for generations (since 1914 to be exact)!,en
840602,LeighAnnTowne,"Grand Rapids, Michigan","Host FOX 17 Morning Mix, jokester, DIY-er, thrifter, craft beer nut, loves Bloody Mary's, Bronson's momhttps://www.facebook.com/leighann.towne/",en
593572,ALBrutel,Paris,NaN,unknown
1053450,Fonkwadiour1,"TOUBA, SENEGAL",BAYE FALL,de
1175310,JUSTINAGROSSO,NaN,NaN,unknown
1096020,SarahRadif,"Southampton, England",Associate Professor Professional Practice Head of Science and Engineering @SolentUni,en
619479,CarineMayol,"Grenoble, Frankreich",NaN,unknown
118673,ManonMondou,Lille,Chargée de #Publicité #Communication / From #Tours born in #Chicago / Lives in #Lille,fr
948480,erickleangar,Venezuela,NaN,unknown
1002694,elaineguimarae,"Petrópolis, RJ",Bibliotecária,es


In [84]:


true_labels = {
    'MorganJewelers1': 'non_fr',
    'LeighAnnTowne': 'non_fr',
    'ALBrutel': 'NA',
    'Fonkwadiour1': 'non_fr',
    'JUSTINAGROSSO': 'NA',
    'SarahRadif': 'non_fr',
    'CarineMayol': 'NA',
    'ManonMondou': 'fr',
    'erickleangar': 'NA',
    'elaineguimarae': 'non_fr',
    'danyunes': 'fr',
    'kuronapilled': 'non_fr',
    'KiriYaji': 'non_fr',
    'floriannisolle': 'fr',
    'Naman93726237': 'fr', #french but from guinea
    'doktor_vananga': 'non_fr', 
    'Slaiidh': "non_fr", #wrote non. could be another language
    'StarRaull': "non_fr",
    'SPeperstraete': 'non_fr',
    'yorgaine_lyon69': 'fr',
    'blinameta': 'non_fr',
    'AnnaRus75': 'non_fr',
    'BernTurner': 'NA',
    'lauriebeebe2': 'non_fr',
    'JeremyyPalazzo': 'NA',
    'GalerieCoulange': 'fr',
    'HaziqkaJ': 'NA',
    'DaraCheick': 'non_fr', #but seems to belong to a french uni
    'mmdlbdrny2': 'NA',
    'blandineleonie': 'fr',
    'CarlosO324': 'non_fr',
    'joshuatoney3': 'non_fr',
    'StephaneWayler': 'fr',
    'JessieV_214': 'non_fr',
    'Apartofsuzanne': 'non_fr',#but writes that she is french, and location is Paris
    'katuxka1': 'NA',
    'tamaitanya': 'NA',
    'sulagnasays': 'non_fr',
    'nicolesimon': 'non_fr',
    'lunasitah': 'non_fr',
    'ln971': 'fr',
    'olive_cas': 'non_fr',
    'Mercedeszatfc': 'NA',
    'hady_saad': 'non_fr',
    'GatienLeroux': 'fr',
    'Meluxiam': 'non_fr',
    'yulimadera10': 'non_fr',
    'RyanBrossault': 'fr',
    'FlotillaMRY': 'non_fr',
    'love98_daniela': 'non_fr',
    'jucag4115': 'NA',
    'Bxwiz': 'non_fr',
    'MLECOMTE': 'fr',
    'DelgadoJanahi': 'non_fr',
    'f1rmin': 'fr',
    'SylvainWalczak': 'NA',
    'SoyLigiawn': 'non_fr',
    'javielo26': 'non_fr',
    'CherrybangMUA': 'non_fr',
    '1927_albert': 'non_fr',
    'aayyoud': 'NA',
    'princessivy28': 'non_fr',
    'drumz420': 'NA',
    'Cjruin': 'non_fr',
    'xXallymayXx': 'non_fr',
    'NaAutacaace': 'fr',
    'RosaLuc15360022': 'non_fr',
    'PatrikWinston': 'non_fr',
    'mwa_sisqo': 'NA',
    'vankrug': 'NA',
    'miktoi': 'non_fr',
    'cescoeco': 'non_fr',
    'floo_ncy': 'NA',
    'Quentin_1411': 'non_fr',
    'Tvy_Tk': 'NA',
    'AlbanLeneveu': 'NA',
    'adh2311': 'non_fr',
    'loaizayose': 'NA',
    'mathilde_Fparis': 'fr',
    'gouillardmichae': 'fr',
    'RomainSprynski': 'fr',
    'styledscience': 'non_fr',
    'amhammadi': 'NA',
    'AliRazaSharif2': 'NA',
    'JahanLutz': 'fr',
    'HERVEJEHL': 'fr',
    'CatherineMSch': 'fr',
    'PoliKeyCo': 'non_fr',
    'HisGraceth': 'non_fr',
    'verstagoat': 'non_fr',
    'G_deLinares': 'fr', 
    'Granadechkou': 'fr',
    'MommaDandine': 'NA',
    'allionesolution': 'non_fr',
    'Ross75016': 'fr',
    'DorignyTheo': 'NA',
    'nikkolasg1': 'non_fr',
    'InesNau': 'fr',
    'FrenchEmperior': 'non_fr',
    'BekaEssick': 'non_fr'}

In [87]:
def label_language(lang):
    if lang == 'unknown':
        return 'NA'
    elif lang == 'fr':
        return 'fr'
    else:
        return 'non_fr'

test1['pred_lang'] = test1['language'].apply(label_language)
test2['pred_lang'] = test2['language'].apply(label_language)

test1['true_lang'] = test1['screen_name'].map(true_labels)
test2['true_lang'] = test2['screen_name'].map(true_labels)

In [93]:
test1.head(5)

,follower_id,screen_name,description,timestamp_utc,location,tweets,followers,friends,likes,lists,description_cleantext,language,pred_lang,true_lang
700690,501147199,MorganJewelers1,Morgan Jewelers has been making a difference in families for generations (since 1914 to be exact)!,1.330031e+09,NaN,405.0,2957.0,479.0,17.0,7.0,Morgan Jewelers has been making a difference in families for generations (since 1914 to be exact)!,en,non_fr,non_fr
840602,457164360,LeighAnnTowne,"Host FOX 17 Morning Mix, jokester, DIY-er, thrifter, craft beer nut, loves Bloody Mary's, Bronson's mom\nhttps://www.facebook.com/leighann.towne/",1.325906e+09,"Grand Rapids, Michigan",4497.0,2373.0,2404.0,1221.0,48.0,"Host FOX 17 Morning Mix, jokester, DIY-er, thrifter, craft beer nut, loves Bloody Mary's, Bronson's momhttps://www.facebook.com/leighann.towne/",en,non_fr,non_fr
593572,2922883419,ALBrutel,NaN,1.418652e+09,Paris,644.0,100.0,189.0,187.0,4.0,NaN,unknown,NA,NA
1053450,2422167290,Fonkwadiour1,BAYE FALL,1.396361e+09,"TOUBA, SENEGAL",384.0,38.0,215.0,238.0,0.0,BAYE FALL,de,non_fr,non_fr
1175310,2677456479,JUSTINAGROSSO,NaN,1.404611e+09,NaN,11011.0,218.0,129.0,2429.0,3.0,NaN,unknown,NA,NA


In [94]:
from sklearn.metrics import classification_report, accuracy_score

# Calculate metrics for test1
print("Metrics for test1:")
print(classification_report(test1['true_lang'], test1['pred_lang']))
print("Accuracy:", accuracy_score(test1['true_lang'], test1['pred_lang']))

# Calculate metrics for test2
print("\nMetrics for test2:")
print(classification_report(test2['true_lang'], test2['pred_lang']))
print("Accuracy:", accuracy_score(test2['true_lang'], test2['pred_lang']))

Metrics for test1:
              precision    recall  f1-score   support

          NA       1.00      1.00      1.00        25
          fr       1.00      0.96      0.98        24
      non_fr       0.98      1.00      0.99        51

    accuracy                           0.99       100
   macro avg       0.99      0.99      0.99       100
weighted avg       0.99      0.99      0.99       100

Accuracy: 0.99

Metrics for test2:
              precision    recall  f1-score   support

          NA       1.00      0.96      0.98        25
          fr       1.00      0.92      0.96        24
      non_fr       0.94      1.00      0.97        51

    accuracy                           0.97       100
   macro avg       0.98      0.96      0.97       100
weighted avg       0.97      0.97      0.97       100

Accuracy: 0.97


- Still lack the actual language filtering
- Think of a more sophisticated way to locate french users than only using language. This method should capture french users that write in english, and in french, and avoid selecting users that are writing in french but are not actually french. 